In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#default_exp search

In [3]:
#export
from food.tools import *
from food.paths import *
from food.psql import *
import requests
from food.qdrant import *
import torch
from torch.nn import CosineSimilarity
cos = CosineSimilarity(dim=1, eps=1e-08)
import numpy as np

In [4]:
#export
## temp while api is not aperational
# from food.clipmodel import *
from PIL import Image

In [5]:
# #export
# foods = read_sql('foods')
# # foods = foods.drop(columns = ['clip'])
# foods = foods.set_index('id')

In [6]:
#export
collection_name = 'food_images'
table = 'foods_prompted'
foods = read_sql(table)
foods = foods.set_index('id')

In [7]:
#export
def read_image_from_url(url=None,path=None):
    if url: response = requests.get(url, stream=True)
    if path: pass
    return Image.open(response.raw)



In [8]:
#export
def search_image_(url=None,head = 1,env='dev'):
    client = dev_client if env == 'dev' else prod_client
    image_clip = requests.post(f'https://guru.skynet.center/image2vector/?url={url}').json()
    results = client.search(collection_name=collection_name,query_vector=image_clip,top=head)
    image_clip = torch.Tensor(image_clip)
    df = foods.loc[[r.id for r in results]].copy()
    df['score'] = [r.score for r in results]
    df = df.sort_values('score',ascending=False)

    return image_clip,df.reset_index()

series2tensor = lambda series:torch.tensor([np.array(c) for c in series.values])

In [9]:
#export
drop_vector =      lambda clip, i: torch.cat([clip[0:i], clip[i+1:]])

def multiply_vector(clip,i,n=1):
    duplicated = torch.cat([clip[i].reshape(1,768) for _ in range(n)])
    return torch.cat([clip, duplicated.reshape(n,768)])

In [112]:
#export
def multiple_foods(url,env='dev'):

    image_clip,selected = search_image_(url,head=100,env=env)
    selected=selected.reset_index(drop=True)
    clip = series2tensor(selected['clip'])
    initscore = float(cos(image_clip.reshape(1,768), clip.mean(0).reshape(1,768)).detach().clone())
    startscore = initscore-0.0000001
    n=0

    while startscore !=initscore:
        startscore = initscore
        
        selected = selected.reset_index(drop=True)
        for i in reversed(selected.index):
            clip = series2tensor(selected['clip'])
            dropped = drop_vector(clip,i)
            testscore = float(cos(image_clip.reshape(1,768), dropped.mean(0).reshape(1,768)).detach().clone())

            if testscore > initscore-0.0001: 
                r = selected.loc[i,'text']
                selected = selected.drop(i)
                initscore = testscore

            else:
                extra = multiply_vector(clip,i,1)
                testscore = float(cos(image_clip.reshape(1,768), extra.mean(0).reshape(1,768)).detach().clone())
                if testscore > initscore: 
                    selected = selected.append(selected.loc[i])
                    initscore = testscore

            n+=1
            if n ==15:break

        print(initscore)
    count = selected.groupby('text')['clip'].count().sort_index()
    r = selected.drop_duplicates('text').set_index("text").sort_index()
    r['count'] = count
    r = r.sort_values('count',ascending = False)
    r['cumcount'] = r['count'].cumsum()/r['count'].sum()

    description = '. '.join(r[r['cumcount']<0.9]['description'].unique().tolist())


    return r.drop(columns = ['clip']), description, selected,initscore

In [113]:
#export
def search_image(url,env='dev'):
    r, desc, sel,score = multiple_foods(url,env=env)
    df = sel[['energy','protein','carb','fat','score']].mean().to_frame().T
    df['score'] =score
    df['description'] = desc
    return df.round()


In [123]:
url = 'https://glovo.away.guru/test_photos/3fbd4cbe-478a-4482-99e2-43588541e950.jpeg'

In [120]:
url = 'https://img1.russianfood.com/dycontent/images_upl/465/big_464858.jpg'

In [124]:
r, desc, sel,score = multiple_foods(url,env='prod')

0.2125802515049535
0.23824091537855335
0.2470458392318917
0.25041994825349023
0.25039596514915996
0.25034885881965907
0.25028950119968735
0.25028950119968735


In [125]:
r

,id,description,category,energy,protein,carb,fat,score,count,cumcount
text,,,,,,,,,,
"the food is egg rolls, dumplings, sushi . it has a little bit of topped with salmon sushi",1102354,"Sushi, topped with salmon","Egg rolls, dumplings, sushi",107.0,8.92,13.39,1.74,0.818805,3,0.333333
the food is fish . it has a little bit of steamed or poached salmon,1098977,"Salmon, steamed or poached",Fish,160.0,25.78,0.00,5.53,0.733792,3,0.666667
the food is fish . it has a little bit of made with margarine baked or broiled salmon,1098964,"Salmon, baked or broiled, made with margarine",Fish,177.0,25.07,0.13,7.84,0.755125,2,0.888889
"the food is egg rolls, dumplings, sushi . it has a little bit of california sushi roll",1102346,"Sushi roll, California","Egg rolls, dumplings, sushi",93.0,2.92,18.41,0.67,0.826967,1,1.000000


In [16]:
search_image(url,env='prod')

,energy,protein,carb,fat,score,description
0,131.0,12.0,7.0,6.0,0.0,"Beef, oxtails, cooked. Mushrooms, fresh, cooke..."


In [5]:
!nbdev_build_lib

Converted 00_clipmodel.ipynb.
Converted 00_custom_pandas.ipynb.
Converted 00_paths.ipynb.
Converted 00_progress_check.ipynb.
Converted 00_psql.ipynb.
Converted 00_qdrant.ipynb.
Converted 00_tools.ipynb.
Converted 01_multiple_foods.ipynb.
Converted 01_multiple_foods_segmantation.ipynb.
Converted 01_search.ipynb.
Converted 0_template copy 2.ipynb.
Converted 0_template copy.ipynb.
Converted 0_template.ipynb.
Converted OFA.ipynb.
Converted Untitled-1.ipynb.
Converted bot pseudocode.ipynb.
Converted bot_test.ipynb.
Converted classifying_glovo_images.ipynb.
Converted foodd dataset.ipynb.
Converted foods_prompted_tosql.ipynb.
Converted ideas.ipynb.
Converted inference.ipynb.
Converted multiple3105.ipynb.
Converted multiple_3005.ipynb.
No export destination, ignored:
#export
def search_image(url=None,head = 1):
    image_clip = requests.post(f'https://guru.skynet.center/image2vector/?url={url}').json()
    results = client.search(collection_name=collection_name,query_vector=image_clip,top=head